In [1]:
# Install sklearn
%pip install scikit-learn==1.5.2

# Install TabPFN
%pip install tabpfn

%pip install git
# TabPFN Community installs optional functionalities around the TabPFN model
# These include post-hoc ensembles, interpretability tools, and more

# Install Baselines
%pip install catboost xgboost

# Install example datasets
%pip install datasets

%pip install  tabpfn_extensions
%pip install torch==2.1.2+cu121 torchvision torchaudio -f https://mirrors.aliyun.com/pytorch-wheels/cu121/

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simpleNote: you may need to restart the kernel to use updated packages.

Could not fetch URL https://pypi.tuna.tsinghua.edu.cn/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.tuna.tsinghua.edu.cn', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate (_ssl.c:1147)'))) - skipping
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Could not fetch URL https://pypi.tuna.tsinghua.edu.cn/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.tuna.tsinghua.edu.cn', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate (_ssl.c:1147)'))) - skipping
Note: you may need to restart th

ERROR: Could not find a version that satisfies the requirement git (from versions: none)
ERROR: No matching distribution found for git


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Could not fetch URL https://pypi.tuna.tsinghua.edu.cn/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.tuna.tsinghua.edu.cn', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate (_ssl.c:1147)'))) - skipping
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simpleNote: you may need to restart the kernel to use updated packages.

Could not fetch URL https://pypi.tuna.tsinghua.edu.cn/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.tuna.tsinghua.edu.cn', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certifi

In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from IPython.display import display, Markdown, Latex
from tabpfn import TabPFNClassifier, TabPFNRegressor
from sklearn.metrics import (
    accuracy_score,
    mean_absolute_error,
    mean_squared_error,
    root_mean_squared_error,
    r2_score,
    roc_auc_score,
)
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import cross_val_score
from tabpfn_extensions.post_hoc_ensembles.sklearn_interface import AutoTabPFNClassifier, AutoTabPFNRegressor
import torch
if not torch.cuda.is_available():
    raise SystemError('GPU device not found. For fast training, please enable GPU. See section above for instructions.')
from tabpfn_extensions import unsupervised
from sklearn.datasets import load_breast_cancer
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.inspection import DecisionBoundaryDisplay
from AutoClean import AutoClean
import pandas as pd
from imblearn.over_sampling import SMOTE
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

In [16]:


# 文件路径列表
file_paths = [
    'UNSW_2018_IoT_Botnet_Full5pc_1.csv',
    'UNSW_2018_IoT_Botnet_Full5pc_2.csv',
    'UNSW_2018_IoT_Botnet_Full5pc_4.csv'
]

# 读取所有CSV文件并存储在列表中
dfs = [pd.read_csv(file) for file in file_paths]

# 合并所有的数据框（默认按行合并）
combined_df = pd.concat(dfs, ignore_index=True)

# 获取倒数三列的列名
last_three_columns = combined_df.columns[-3:]

# 遍历倒数三列，输出每列的类别及各类别的数量
for column in last_three_columns:
    print(f"Column: {column}")
    print(combined_df[column].value_counts())  # 输出类别及其数量
    print("-" * 30)  # 分隔线

Column: attack
attack
1    2668045
0        477
Name: count, dtype: int64
------------------------------
Column: category
category
DoS               1650260
DDoS               926624
Reconnaissance      91082
Normal                477
Theft                  79
Name: count, dtype: int64
------------------------------
Column: subcategory
subcategory
UDP                  1609859
TCP                   964551
Service_Scan           73168
OS_Fingerprint         17914
HTTP                    2474
Normal                   477
Keylogging                73
Data_Exfiltration          6
Name: count, dtype: int64
------------------------------


原始数据情况 
Column: attack
attack
1    2668045
0        477
Name: count, dtype: int64
------------------------------
Column: category
category
DoS               1650260
DDoS               926624
Reconnaissance      91082
Normal                477
Theft                  79
Name: count, dtype: int64
------------------------------
Column: subcategory
subcategory
UDP                  1609859
TCP                   964551
Service_Scan           73168
OS_Fingerprint         17914
HTTP                    2474
Normal                   477
Keylogging                73
Data_Exfiltration          6
Name: count, dtype: int64
------------------------------

In [17]:
# ----------- 新增：过滤掉 IPv6 地址所在的行 -----------
def is_valid_ipv4(ip):
    """判断是否为合法 IPv4 地址"""
    if not isinstance(ip, str):
        return False
    parts = ip.split('.')
    if len(parts) != 4:
        return False
    try:
        return all(0 <= int(part) <= 255 for part in parts)
    except ValueError:
        return False

# 检查 saddr 和 daddr 是否都是 IPv4，如果不是，就过滤整行
if 'saddr' in combined_df.columns and 'daddr' in combined_df.columns:
    combined_df = combined_df[
        combined_df['saddr'].apply(is_valid_ipv4) &
        combined_df['daddr'].apply(is_valid_ipv4)
    ]
elif 'saddr' in combined_df.columns:
    combined_df = combined_df[combined_df['saddr'].apply(is_valid_ipv4)]
elif 'daddr' in combined_df.columns:
    combined_df = combined_df[combined_df['daddr'].apply(is_valid_ipv4)]

print("✅ 已过滤包含 IPv6 的行，仅保留 IPv4 数据")

# ----------- 独热编码部分 -------------
encoded_flgs = pd.get_dummies(combined_df['flgs'], prefix='flgs')
encoded_proto = pd.get_dummies(combined_df['proto'], prefix='proto')
encoded_state = pd.get_dummies(combined_df['state'], prefix='state')

combined_df = pd.concat([combined_df, encoded_flgs, encoded_proto, encoded_state], axis=1)
combined_df.drop(columns=['flgs', 'proto', 'state'], inplace=True)

# ----------- IP地址分割成二进制列 -------------
def ip_to_binary_parts(ip):
    parts = ip.split('.')
    return [format(int(part), '08b') for part in parts]

# 处理 saddr
if 'saddr' in combined_df.columns:
    saddr_binary = combined_df['saddr'].apply(lambda x: pd.Series(ip_to_binary_parts(x)))
    saddr_binary.columns = ['saddr_1', 'saddr_2', 'saddr_3', 'saddr_4']
    combined_df = pd.concat([combined_df, saddr_binary], axis=1)
    combined_df.drop(columns=['saddr'], inplace=True)

# 处理 daddr
if 'daddr' in combined_df.columns:
    daddr_binary = combined_df['daddr'].apply(lambda x: pd.Series(ip_to_binary_parts(x)))
    daddr_binary.columns = ['daddr_1', 'daddr_2', 'daddr_3', 'daddr_4']
    combined_df = pd.concat([combined_df, daddr_binary], axis=1)
    combined_df.drop(columns=['daddr'], inplace=True)

# ----------- 输出结果并保存文件 -------------
print("Processed columns:", combined_df.columns)
combined_df.to_csv('processed_data.csv', index=False)
print("✅ 数据处理完成，已保存到 processed_data.csv")

✅ 已过滤包含 IPv6 的行，仅保留 IPv4 数据
Processed columns: Index(['pkSeqID', 'stime', 'flgs_number', 'proto_number', 'sport', 'dport',
       'pkts', 'bytes', 'state_number', 'ltime', 'seq', 'dur', 'mean',
       'stddev', 'sum', 'min', 'max', 'spkts', 'dpkts', 'sbytes', 'dbytes',
       'rate', 'srate', 'drate', 'TnBPSrcIP', 'TnBPDstIP', 'TnP_PSrcIP',
       'TnP_PDstIP', 'TnP_PerProto', 'TnP_Per_Dport', 'AR_P_Proto_P_SrcIP',
       'AR_P_Proto_P_DstIP', 'N_IN_Conn_P_DstIP', 'N_IN_Conn_P_SrcIP',
       'AR_P_Proto_P_Sport', 'AR_P_Proto_P_Dport',
       'Pkts_P_State_P_Protocol_P_DestIP', 'Pkts_P_State_P_Protocol_P_SrcIP',
       'attack', 'category', 'subcategory', 'flgs_e', 'flgs_e   t',
       'flgs_e  D', 'flgs_e &', 'flgs_e *', 'flgs_e d', 'flgs_e g', 'flgs_e s',
       'flgs_eU', 'proto_arp', 'proto_icmp', 'proto_tcp', 'proto_udp',
       'state_ACC', 'state_CON', 'state_ECO', 'state_FIN', 'state_INT',
       'state_MAS', 'state_REQ', 'state_RST', 'state_TST', 'state_URP',
       'saddr_1', 

In [ ]:
# 定义一个函数，用于将16进制数据转换为10进制
def convert_hex_to_int(value):
    try:
        # 检查是否为16进制格式（以0x开头）
        if isinstance(value, str) and value.startswith('0x'):
            return int(value, 16)  # 将16进制字符串转换为10进制整数
        # 如果已经是整数或其他类型，直接返回
        return int(value)
    except ValueError:
        # 如果转换失败，返回原始值（可根据需要抛出异常或处理错误）
        return value

# 对 dport 和 sport 列应用转换函数
combined_df['dport'] = combined_df['dport'].apply(convert_hex_to_int)
combined_df['sport'] = combined_df['sport'].apply(convert_hex_to_int)

# 检查转换后的结果
print(combined_df.head())
combined_df.to_csv('processed_data_with_converted_ports.csv', index=False)

print("处理完成，结果已保存到 processed_data_with_converted_ports.csv")

   pkSeqID         stime  flgs_number  proto_number  sport  dport  pkts  \
0        1  1.528089e+09            1             1  49960     80     8   
1        2  1.528089e+09            1             2     -1     -1     2   
2        3  1.528089e+09            1             1  49962     80     8   
3        4  1.528089e+09            1             1  49964     80     8   
4        5  1.528089e+09            1             1  49966     80     8   

   bytes  state_number         ltime  ...  state_TST  state_URP   saddr_1  \
0   1980             1  1.528089e+09  ...      False      False  11000000   
1    120             2  1.528089e+09  ...      False      False  11000000   
2   2126             1  1.528089e+09  ...      False      False  11000000   
3   2024             1  1.528089e+09  ...      False      False  11000000   
4   2319             1  1.528089e+09  ...      False      False  11000000   

    saddr_2   saddr_3   saddr_4   daddr_1   daddr_2   daddr_3   daddr_4  
0  10101000 

In [19]:
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# 分离特征和目标变量
drop_columns = ['attack', 'category', 'subcategory','stime','pkSeqID']
target_columns = ['category']
X = combined_df.drop(columns=drop_columns, errors='ignore')  # 特征
y = combined_df[target_columns].values.ravel()  # 目标变量 (展平为一维数组)

# 检查原始数据的目标变量分布
print("Original distribution of category:")
print(Counter(y))
print("-" * 30)

# 定义采样目标数量（最终各类都需要到这个数量）
target_sample_size = 5000

# 创建字典，定义每类的采样策略
original_distribution = Counter(y)
sampling_strategy_over = {}
sampling_strategy_under = {}

for cls, count in original_distribution.items():
    if count < target_sample_size:
        # 需要过采样的类别
        sampling_strategy_over[cls] = target_sample_size
    elif count > target_sample_size:
        # 需要欠采样的类别
        sampling_strategy_under[cls] = target_sample_size

# 定义过采样和欠采样步骤
over = SMOTE(sampling_strategy=sampling_strategy_over, random_state=42)
under = RandomUnderSampler(sampling_strategy=sampling_strategy_under, random_state=42)

# 将采样方法组合成一个管道
pipeline = Pipeline(steps=[('o', over), ('u', under)])

# 应用采样管道
X_resampled, y_resampled = pipeline.fit_resample(X, y)

# 检查采样后的分布
print("Resampled distribution of category:")
print(Counter(y_resampled))
print("-" * 30)
# 将采样后的特征和目标变量合并为一个 DataFrame
resampled_df = pd.DataFrame(X_resampled, columns=X.columns)  # 采样后的特征
resampled_df['category'] = y_resampled  # 添加采样后的目标变量

# 输出为 Excel 文件
output_file = "resampled_data.csv"
resampled_df.to_csv(output_file, index=False)

print(f"Resampled data has been saved to {output_file}")

Original distribution of category:
Counter({'DoS': 1650260, 'DDoS': 926624, 'Reconnaissance': 91082, 'Normal': 455, 'Theft': 79})
------------------------------


KeyboardInterrupt: 

In [20]:
import pandas as pd

# 假设 combined_df 是你的原始 DataFrame
# 目标变量列名为 'category'

# 定义采样策略
target_sample_size = {
    'DoS': 20000,  # 随机选取 20000 个样本
    'DDoS': 20000,  # 随机选取 10000 个样本
    'Reconnaissance': 10000,  # 随机选取 10000 个样本
    'Normal': None,  # 全部选取
    'Theft': None   # 全部选取
}

# 按照类别分组并采样
def sample_by_category(df, target_column, sample_strategy):
    sampled_dfs = []
    
    for category, sample_size in sample_strategy.items():
        # 获取当前类别的子集
        category_subset = df[df[target_column] == category]
        
        if sample_size is None:
            # 如果 sample_size 为 None，则选取全部
            sampled_dfs.append(category_subset)
        else:
            # 否则随机抽样指定数量
            if len(category_subset) < sample_size:
                print(f"Warning: Category '{category}' has fewer samples ({len(category_subset)}) than requested ({sample_size}). Selecting all.")
            sampled_dfs.append(category_subset.sample(n=sample_size, random_state=42))
    
    # 将所有采样后的子集合并
    return pd.concat(sampled_dfs, axis=0)

# 应用采样函数
resampled_df = sample_by_category(combined_df, target_column='category', sample_strategy=target_sample_size)

# 检查采样后的分布
print("Resampled distribution of category:")
print(resampled_df['category'].value_counts())
print("-" * 30)

# 输出为 CSV 文件
output_file = "resampled_data.csv"
resampled_df.to_csv(output_file, index=False)

print(f"Resampled data has been saved to {output_file}")

Resampled distribution of category:
category
DoS               20000
DDoS              20000
Reconnaissance    10000
Normal              455
Theft                79
Name: count, dtype: int64
------------------------------
Resampled data has been saved to resampled_data.csv
